In [5]:
pip install lgm

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.2 MB/s eta 0:00:00:00:01
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [10]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 100)
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from lightgbm import LGBMRegressor


s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

file_key_1 = 'Tabular-Playground-Series/Tabular-Playground-Jan-2023-1/train.csv'
file_key_2 = 'Tabular-Playground-Series/Tabular-Playground-Jan-2023-1/test.csv'
file_key_3 = 'Tabular-Playground-Series/Tabular-Playground-Jan-2023-1/sample_submission.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

## Reading data files
train = pd.read_csv(file_content_stream_1)
test = pd.read_csv(file_content_stream_2)
submission = pd.read_csv(file_content_stream_3)

In [12]:
X = train.drop(columns = ['id', 'MedHouseVal'], axis = 1)
Y = train['MedHouseVal']

# scaler = MinMaxScaler()

## Defining list to store results
lgb_results = list()

for i in range(0, 5):
    
    kfold = KFold(n_splits = 5, shuffle = True, random_state = i)
    
    ## Defining list to store fold results
    lgb_fold_result = list()
    
    for train_ix, test_ix in kfold.split(X, Y):
    
        ## Splitting the data 
        X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
        Y_train, Y_test = Y.iloc[train_ix], Y.iloc[test_ix]
        
        ## Scaling the data
#         X_train = scaler.fit_transform(X_train)
#         X_test = scaler.fit_transform(X_test)
        
        ## Building the model 
        lgb_md = LGBMRegressor(n_estimators = 1000, 
                                learning_rate = 0.01,
                                num_leaves = 40,
                                max_depth = 15, 
                                lambda_l1 = 3, 
                                lambda_l2 = 1, 
                                bagging_fraction = 0.95, 
                                feature_fraction = 0.96).fit(X_train, Y_train)

        ## Predicting on test
        lgb_pred = lgb_md.predict(X_test)
        
        ## Appending fold performance
        lgb_fold_result.append(mean_squared_error(Y_test, lgb_pred))
                               
    ## Appending average mse k-fold run 
    lgb_results.append(np.mean(lgb_fold_result))
    
print('The average mse of k-NN over 5-fold CV (running 5 times) is', np.mean(lgb_results))

[LightGBM] [Warning] lambda_l1 is set=3, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3
[LightGBM] [Warning] feature_fraction is set=0.96, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.96
[LightGBM] [Warning] lambda_l2 is set=1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1
[LightGBM] [Warning] bagging_fraction is set=0.95, subsample=1.0 will be ignored. Current value: bagging_fraction=0.95
[LightGBM] [Warning] lambda_l1 is set=3, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3
[LightGBM] [Warning] feature_fraction is set=0.96, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.96
[LightGBM] [Warning] lambda_l2 is set=1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1
[LightGBM] [Warning] bagging_fraction is set=0.95, subsample=1.0 will be ignored. Current value: bagging_fraction=0.95
[LightGBM] [Warning] lambda_l1 is set=3, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3
[LightGBM] [W

/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/ec2-user/anaconda3/envs/python3/lib/python3.8/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [13]:
print('The average mse of k-NN over 5-fold CV (running 5 times) is', np.mean(lgb_results))

The average mse of k-NN over 5-fold CV (running 5 times) is 0.3201097070012035


In [15]:
np.sqrt(0.3201097070012035)

0.565782384845272

In [16]:
X = train.drop(columns = ['id', 'MedHouseVal'], axis = 1)
Y = train['MedHouseVal']

X_test = test.drop(columns = 'id', axis = 1) 

## Scaling the input variables
# X = scaler.fit_transform(X)
# X_test = scaler.fit_transform(X_test)

## Building the lightgbm model
lbg_md = LGBMRegressor(n_estimators = 1000, 
                                learning_rate = 0.01,
                                num_leaves = 40,
                                max_depth = 15, 
                                lambda_l1 = 3, 
                                lambda_l2 = 1, 
                                bagging_fraction = 0.95, 
                                feature_fraction = 0.96).fit(X_train, Y_train)

## Predicting 
submission['MedHouseVal'] = lbg_md.predict(X_test)
submission.head()

[LightGBM] [Warning] lambda_l1 is set=3, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3
[LightGBM] [Warning] feature_fraction is set=0.96, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.96
[LightGBM] [Warning] lambda_l2 is set=1, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1
[LightGBM] [Warning] bagging_fraction is set=0.95, subsample=1.0 will be ignored. Current value: bagging_fraction=0.95


,id,MedHouseVal
0,37137,0.701803
1,37138,1.013380
2,37139,4.137471
3,37140,3.370196
4,37141,2.584709


In [ ]:
submi